In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
# This is needed to display the images.
%matplotlib inline

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

In [4]:
# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

In [5]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

In [6]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [7]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [8]:
from pykinect2 import PyKinectV2
from pykinect2.PyKinectV2 import *
from pykinect2 import PyKinectRuntime
from acquisitionKinect import AcquisitionKinect
from frame import Frame as Frame
import numpy as np
import cv2
import ctypes
import _ctypes
import sys
import face_recognition
import os

pykinect = PyKinectRuntime.PyKinectRuntime(PyKinectV2.FrameSourceTypes_Depth)
kinect = AcquisitionKinect()
f = Frame()

known_face_encodings = []
known_face_names = []

# Load sample pictures and learn how to recognize it.
dirname = 'face_recognition/knowns'
files = os.listdir(dirname)
for filename in files:
    name, ext = os.path.splitext(filename)
    if ext == '.jpg':
        known_face_names.append(name)
        pathname = os.path.join(dirname, filename)
        img = face_recognition.load_image_file(pathname)
        face_encoding = face_recognition.face_encodings(img)[0]
        known_face_encodings.append(face_encoding)

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    while True:
        
        # --- Getting frames and drawing
        kinect.get_frame(f)
        kinect.get_color_frame()
        image_np = kinect._frameRGB
        image_np = cv2.cvtColor(image_np, cv2.COLOR_RGBA2RGB)

        # ----- Face Recognition
        # Resize frame of video to 1/4 size for faster face recognition processing
        rgb_small_frame = cv2.resize(image_np, (0, 0), fx=0.25, fy=0.25)

        # Only process every other frame of video to save time
        if process_this_frame:
            # Find all the faces and face encodings in the current frame of video
            face_locations = face_recognition.face_locations(rgb_small_frame)
            face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

            face_names = []
            for face_encoding in face_encodings:
                # See if the face is a match for the known face(s)
                distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                min_value = min(distances)

                # tolerance: How much distance between faces to consider it a match. Lower is more strict.
                # 0.6 is typical best performance.
                name = "Unknown"
                if min_value < 0.6:
                    index = np.argmin(distances)
                    name = known_face_names[index]

                face_names.append(name)

        process_this_frame = not process_this_frame

        # Display the results
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            # Scale back up face locations since the frame we detected in was scaled to 1/4 size
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4

            # Draw a box around the face
            cv2.rectangle(image_np, (left, top), (right, bottom), (0, 0, 255), 2)
            # Draw a label with a name below the face
            cv2.rectangle(image_np, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(image_np, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)


        # ----- Object Recognition
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')

        # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image_np, axis=0)
        # Actual detection.
        (boxes, scores, classes, num) = sess.run(
            [detection_boxes, detection_scores, detection_classes, num_detections],
            feed_dict={image_tensor: image_np_expanded})
        # Visualization of the results of a detection.
        vis_util.visualize_boxes_and_labels_on_image_array(
            image_np,
            np.squeeze(boxes),
            np.squeeze(classes).astype(np.int32),
            np.squeeze(scores),
            category_index,
            use_normalized_coordinates=True,
            line_thickness=8)

        # Get coordinates of detected boxes ; ymin, ymax, xmin, xmax
        coordinates = vis_util.return_coordinates(
                    image_np,
                    np.squeeze(boxes),
                    np.squeeze(classes).astype(np.int32),
                    np.squeeze(scores),
                    category_index,
                    use_normalized_coordinates=True,
                    line_thickness=8,
                    min_score_thresh=0.60)
        if coordinates is not None:

            # ----- Face Recognition
            print(face_names)
            count = 0
            for i in range(len(coordinates)):
                if coordinates[i][0] == 'person':
                    if count < len(face_names):
                        coordinates[i][0] = face_names[count]
                        count += 1

            #print(*coordinates)
            print(coordinates)
            xpoint_for_depth=[]
            ypoint_for_depth=[]

            print("object 개수:", len(coordinates))

            # Connect Color and Depth Info.
            frame = pykinect.get_last_depth_frame()
            frameD = pykinect._depth_frame_data
            frame = frame.astype(np.uint8)
            frame = np.reshape(frame, (424, 512))
            frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)

            # Take just one point : middle
            '''for i in range(len(coordinates)):

                ypoint_for_depth.append(int((coordinates[i][1]+coordinates[i][2])/2))
                xpoint_for_depth.append(int((coordinates[i][3]+coordinates[i][4])/2))

            print("ypoint_for_depth:", ypoint_for_depth)
            print("xpoint_for_depth:", xpoint_for_depth)

            ##output = cv2.bilateralFilter(output, 1, 150, 75)
            cv2.imshow('KINECT Video Stream', frame)

            for i in range(len(coordinates)):
                Pixel_Depth = frameD[((ypoint_for_depth[i] * 512) + xpoint_for_depth[i])]
                print("점 하나만 고려:",i+1, "번째 object의 depth value: ", Pixel_Depth)'''

            # Get average depth value considering all coordinate pairs. (List Version)
            '''Pixel_Depth_List = []
            Pixel_Depth_Sum = 0
            non_error_count = 0
            for i in range(len(coordinates)):
                for x in range(int(coordinates[i][3]), int(coordinates[i][4]+1)):
                    for y in range(int(coordinates[i][1]), int(coordinates[i][2]+1)):
                        Pixel_Depth_Sum += frameD[((y * 512) + x)]
                        if frameD[((y * 512) + x)] != 0:
                            non_error_count += 1

                Pixel_Depth_List.append(int(Pixel_Depth_Sum/non_error_count))

            for i in range(len(coordinates)):
                print("전체 면적 고려:", i+1, "번째 object의 depth value: ", Pixel_Depth_List[i])
            print("\n")'''

            # Get average depth value considering all coordinate pairs. (Dict Version)
            Pixel_Depth_Dict = {}
            Pixel_Depth_Sum = 0
            non_error_count = 0
            for i in range(len(coordinates)):

                # ----- Depth Recognition
                for x in range(int(coordinates[i][3]), int(coordinates[i][4]+1)):
                    for y in range(int(coordinates[i][1]), int(coordinates[i][2]+1)):
                        Pixel_Depth_Sum += frameD[((y * 512) + x)]
                        if frameD[((y * 512) + x)] != 0:
                            non_error_count += 1

                Pixel_Depth_Dict[coordinates[i][0]] = int(Pixel_Depth_Sum/non_error_count)

            # Compare each objects' depth. (Sorting: close -> far)
            sorted(Pixel_Depth_Dict.items(), key=lambda t : t[1])
            print(Pixel_Depth_Dict)
            print("\n")

            # VideoFileOutput.write(image_np)
            cv2.imshow('KINECT Color', image_np)
            cv2.waitKey(1)


        key = cv2.waitKey(1)
        if key == 27: break


['SoraRyu']
[['SoraRyu', 180, 771, 902, 1470, 94.96618509292603]]
object 개수: 1
{'SoraRyu': 2025}


['SoraRyu']
[['SoraRyu', 174, 774, 896, 1491, 95.19542455673218]]
object 개수: 1
{'SoraRyu': 2029}


['SoraRyu']
[['SoraRyu', 176, 772, 896, 1494, 97.45906591415405]]
object 개수: 1
{'SoraRyu': 2031}


['SoraRyu']
[['SoraRyu', 173, 775, 898, 1485, 93.45501661300659]]
object 개수: 1
{'SoraRyu': 2025}


['SoraRyu']
[['SoraRyu', 169, 778, 895, 1512, 95.2855110168457]]
object 개수: 1
{'SoraRyu': 1890}


['SoraRyu']
[['SoraRyu', 104, 742, 857, 1534, 87.0255708694458], ['person', 117, 795, 778, 1829, 68.0982232093811]]
object 개수: 2
{'SoraRyu': 1873, 'person': 1874}


['SoraRyu']
[['SoraRyu', 109, 760, 862, 1907, 83.56216549873352], ['bed', 622, 1075, 8, 1865, 61.66056990623474]]
object 개수: 2
{'SoraRyu': 1888, 'bed': 12130}


['SoraRyu']
[['SoraRyu', 115, 762, 871, 1920, 87.3709499835968]]
object 개수: 1
{'SoraRyu': 1885}


['SoraRyu']
[['SoraRyu', 113, 761, 870, 1920, 84.73198413848877]]
object 개수: 1
{'S

{'SoraRyu': 1619}


['SoraRyu']
[['SoraRyu', 23, 742, 674, 1262, 98.15856218338013]]
object 개수: 1
{'SoraRyu': 1643}


['SoraRyu']
[['SoraRyu', 16, 740, 686, 1271, 98.41529130935669]]
object 개수: 1
{'SoraRyu': 1620}


['SoraRyu']
[['SoraRyu', 21, 741, 683, 1268, 97.62094616889954]]
object 개수: 1
{'SoraRyu': 1620}


['SoraRyu']
[['SoraRyu', 25, 739, 684, 1276, 97.12780117988586]]
object 개수: 1
{'SoraRyu': 1605}


['SoraRyu']
[['SoraRyu', 20, 742, 687, 1266, 98.51025342941284]]
object 개수: 1
{'SoraRyu': 1625}


['SoraRyu']
[['SoraRyu', 20, 742, 685, 1272, 98.06371927261353]]
object 개수: 1
{'SoraRyu': 1618}


['SoraRyu']
[['SoraRyu', 23, 745, 687, 1271, 98.74802827835083]]
object 개수: 1
{'SoraRyu': 1613}


['SoraRyu']
[['SoraRyu', 18, 737, 680, 1272, 98.05271625518799]]
object 개수: 1
{'SoraRyu': 1624}


['SoraRyu']
[['SoraRyu', 19, 739, 688, 1274, 98.39084148406982]]
object 개수: 1
{'SoraRyu': 1613}


['SoraRyu']
[['SoraRyu', 22, 749, 681, 1279, 98.38497638702393]]
object 개수: 1
{'SoraRyu': 1600}




{'SoraRyu': 1430, 'person': 1345}


['SoraRyu', 'EunjungOh']
[['SoraRyu', 11, 768, 699, 1376, 79.2004406452179]]
object 개수: 1
{'SoraRyu': 1356}


['SoraRyu', 'EunjungOh']
[['SoraRyu', 9, 766, 689, 1371, 80.32534122467041]]
object 개수: 1
{'SoraRyu': 1349}


['SoraRyu', 'EunjungOh']
[['SoraRyu', 11, 770, 690, 1359, 82.87172317504883]]
object 개수: 1
{'SoraRyu': 1262}


['SoraRyu', 'EunjungOh']
[['SoraRyu', 8, 767, 684, 1383, 76.73264145851135]]
object 개수: 1
{'SoraRyu': 1282}


['SoraRyu', 'EunjungOh']
[['SoraRyu', 4, 767, 687, 1366, 76.85680389404297]]
object 개수: 1
{'SoraRyu': 1269}


['SoraRyu', 'EunjungOh']
[['SoraRyu', 6, 767, 680, 1385, 77.16608047485352]]
object 개수: 1
{'SoraRyu': 1333}


['SoraRyu', 'EunjungOh']
[['SoraRyu', 277, 729, 1449, 1809, 85.13548374176025], ['EunjungOh', 10, 770, 702, 1341, 78.76277565956116]]
object 개수: 2
{'SoraRyu': 1370, 'EunjungOh': 1283}


['SoraRyu', 'EunjungOh']
[['SoraRyu', 269, 736, 1465, 1823, 85.62890887260437], ['EunjungOh', 10, 769, 699, 1365, 80.

['SoraRyu']
[['SoraRyu', 20, 779, 699, 1920, 95.73663473129272]]
object 개수: 1
{'SoraRyu': 1505}


['SoraRyu']
[['SoraRyu', 22, 778, 703, 1920, 95.87885737419128]]
object 개수: 1
{'SoraRyu': 1506}


['SoraRyu']
[['SoraRyu', 18, 780, 700, 1920, 95.45813202857971]]
object 개수: 1
{'SoraRyu': 1508}


['SoraRyu']
[['SoraRyu', 18, 779, 696, 1920, 95.5237865447998]]
object 개수: 1
{'SoraRyu': 1507}


['SoraRyu']
[['SoraRyu', 19, 780, 696, 1920, 95.41060328483582]]
object 개수: 1
{'SoraRyu': 1506}


['SoraRyu']
[['SoraRyu', 21, 778, 696, 1920, 95.58987617492676]]
object 개수: 1
{'SoraRyu': 1503}


['SoraRyu']
[['SoraRyu', 20, 781, 702, 1920, 95.50106525421143]]
object 개수: 1
{'SoraRyu': 1508}


['SoraRyu']
[['SoraRyu', 20, 783, 702, 1920, 95.60607671737671]]
object 개수: 1
{'SoraRyu': 1509}


['SoraRyu']
[['SoraRyu', 19, 782, 703, 1920, 95.56214809417725]]
object 개수: 1
{'SoraRyu': 1512}


['SoraRyu']
[['SoraRyu', 22, 780, 701, 1920, 95.95208168029785]]
object 개수: 1
{'SoraRyu': 1504}


['SoraRyu']
[['SoraRy

{'person': 1537}


[]
[['person', 37, 780, 720, 1920, 94.36374306678772]]
object 개수: 1
{'person': 1538}


[]
[['person', 37, 791, 727, 1920, 93.77297163009644]]
object 개수: 1
{'person': 1541}


[]
[['person', 39, 791, 717, 1920, 92.5546944141388]]
object 개수: 1
{'person': 1534}


[]
[['person', 38, 784, 723, 1920, 93.4065043926239]]
object 개수: 1
{'person': 1538}


[]
[['person', 35, 797, 716, 1920, 90.29127359390259]]
object 개수: 1
{'person': 1539}


[]
[['person', 38, 791, 714, 1920, 92.32534170150757]]
object 개수: 1
{'person': 1533}


[]
[['person', 36, 796, 715, 1920, 93.27264428138733]]
object 개수: 1
{'person': 1535}


[]
[['person', 34, 790, 706, 1920, 93.28416585922241]]
object 개수: 1
{'person': 1536}


[]
[['person', 37, 787, 717, 1920, 93.31831932067871]]
object 개수: 1
{'person': 1535}


[]
[['person', 38, 794, 713, 1920, 91.4122462272644]]
object 개수: 1
{'person': 1533}


[]
[['person', 38, 787, 716, 1920, 91.54016971588135]]
object 개수: 1
{'person': 1533}


[]
[['person', 35, 796, 716

...

{'person': 1411}


[]
[['person', 12, 778, 25, 1904, 76.94171071052551]]
object 개수: 1
{'person': 1534}


[]
[]
object 개수: 0
{}


[]
[['person', 34, 858, 96, 1605, 77.6436984539032]]
object 개수: 1
{'person': 2952}


[]
[['person', 40, 879, 39, 1601, 83.46242904663086]]
object 개수: 1
{'person': 4513}


[]
[['person', 33, 887, 43, 1576, 85.0506067276001]]
object 개수: 1
{'person': 5097}


[]
[['person', 37, 853, 62, 1571, 82.29989409446716]]
object 개수: 1
{'person': 2413}


[]
[['person', 11, 791, 14, 1915, 70.0312852859497]]
object 개수: 1
{'person': 1400}


[]
[['person', 14, 779, 23, 1902, 75.22419691085815]]
object 개수: 1
{'person': 1430}


[]
[['person', 16, 773, 25, 1902, 74.35646057128906]]
object 개수: 1
{'person': 1435}


[]
[['person', 15, 782, 22, 1903, 70.4264223575592]]
object 개수: 1
{'person': 1444}


[]
[['person', 16, 785, 24, 1901, 72.04256057739258]]
object 개수: 1
{'person': 1434}


[]
[['person', 14, 775, 22, 1906, 74.63668584823608]]
object 개수: 1
{'person': 1427}


[]
[['person', 

{'Unknown': 1376}


['Unknown']
[['Unknown', 16, 797, 21, 1902, 75.34912824630737]]
object 개수: 1
{'Unknown': 1377}


['Unknown']
[['Unknown', 16, 799, 47, 1897, 74.20089840888977]]
object 개수: 1
{'Unknown': 1383}


['Unknown']
[['Unknown', 16, 796, 35, 1897, 75.97242593765259]]
object 개수: 1
{'Unknown': 1376}


['Unknown']
[['Unknown', 17, 775, 88, 1897, 72.48938083648682]]
object 개수: 1
{'Unknown': 1364}


['SoraRyu']
[['SoraRyu', 1, 818, 701, 1915, 87.6872181892395]]
object 개수: 1
{'SoraRyu': 1603}


['SoraRyu']
[['SoraRyu', 9, 815, 679, 1920, 87.249094247818]]
object 개수: 1
{'SoraRyu': 1547}


['SoraRyu']
[['SoraRyu', 9, 810, 694, 1920, 88.0685567855835]]
object 개수: 1
{'SoraRyu': 1543}


['SoraRyu']
[['SoraRyu', 14, 810, 691, 1920, 90.41244983673096]]
object 개수: 1
{'SoraRyu': 1533}


['SoraRyu']
[['SoraRyu', 14, 814, 678, 1920, 87.87003755569458]]
object 개수: 1
{'SoraRyu': 1543}


['SoraRyu']
[['SoraRyu', 0, 762, 697, 1919, 87.50410079956055]]
object 개수: 1
{'SoraRyu': 1556}


['SoraRyu']


{'person': 1378}


[]
[['person', 14, 782, 20, 1920, 77.57036685943604]]
object 개수: 1
{'person': 1381}


[]
[['person', 16, 780, 19, 1920, 78.94560694694519]]
object 개수: 1
{'person': 1378}


[]
[['person', 13, 789, 21, 1920, 78.24685573577881]]
object 개수: 1
{'person': 1386}


[]
[['person', 13, 784, 21, 1920, 79.3003797531128]]
object 개수: 1
{'person': 1385}


[]
[['person', 13, 782, 18, 1920, 79.3251097202301]]
object 개수: 1
{'person': 1385}


[]
[['person', 14, 786, 20, 1920, 78.79118323326111]]
object 개수: 1
{'person': 1386}


[]
[['person', 13, 786, 19, 1920, 80.00832796096802]]
object 개수: 1
{'person': 1384}


[]
[['person', 8, 785, 14, 1920, 78.93218994140625]]
object 개수: 1
{'person': 1392}


[]
[['person', 11, 785, 20, 1920, 79.46890592575073]]
object 개수: 1
{'person': 1390}


[]
[['person', 11, 784, 20, 1920, 80.08641004562378]]
object 개수: 1
{'person': 1392}


[]
[['person', 11, 789, 15, 1920, 78.10590267181396]]
object 개수: 1
{'person': 1396}


[]
[['person', 13, 787, 14, 1920, 80.0

{'SoraRyu': 1569}


['SoraRyu']
[['SoraRyu', 5, 778, 775, 1540, 93.13235282897949]]
object 개수: 1
{'SoraRyu': 1602}


[]
[['person', 3, 768, 765, 1541, 88.70071768760681]]
object 개수: 1
{'person': 1583}


[]
[['person', 2, 768, 768, 1540, 90.55142402648926]]
object 개수: 1
{'person': 1589}


['SoraRyu']
[['SoraRyu', 2, 778, 768, 1541, 92.00822114944458]]
object 개수: 1
{'SoraRyu': 1592}


['SoraRyu']
[['SoraRyu', 3, 780, 769, 1543, 92.15002059936523]]
object 개수: 1
{'SoraRyu': 1593}


[]
[['person', 5, 772, 771, 1537, 88.15761804580688]]
object 개수: 1
{'person': 1592}


[]
[['person', 6, 768, 771, 1537, 90.0149941444397]]
object 개수: 1
{'person': 1588}


['SoraRyu']
[['SoraRyu', 7, 779, 770, 1537, 92.40750670433044]]
object 개수: 1
{'SoraRyu': 1588}


['SoraRyu']
[['SoraRyu', 6, 776, 770, 1535, 90.85702896118164]]
object 개수: 1
{'SoraRyu': 1591}


['SoraRyu']
[['SoraRyu', 3, 775, 767, 1537, 90.00635743141174]]
object 개수: 1
{'SoraRyu': 1590}


['SoraRyu']
[['SoraRyu', 1, 773, 769, 1535, 90.88016152

KeyboardInterrupt: 